In [3]:
!pip install pandas
!pip install bs4
#!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests, time, datetime, re, random

In [5]:
wiki_url =  "https://en.wikipedia.org/wiki/List_of_best-selling_Nintendo_Switch_video_games"
headers = {"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/112.0"}
response = requests.get(wiki_url, headers=headers)
response.status_code

200

In [6]:
# parse data fromt he html into a beautifulsoup object
soup = BeautifulSoup(response.text, "html.parser")
soup

# find the table of switch sales
html_table = soup.find("table", {"class" : "wikitable"})
html_table

# convert the wikipedia table into a pandas dataframe
switch_sales_df = pd.read_html(str(html_table))[0]
switch_sales_df

,Rank,Title,Copies sold,As of,Release date[a],Genre(s),Developer(s),Publisher(s)
0,1,Mario Kart 8 Deluxe,53.79 million[4],"March 31, 2023","April 28, 2017",Kart racing,Nintendo EPD,Nintendo
1,2,Animal Crossing: New Horizons,42.21 million[4],"March 31, 2023","March 20, 2020",Social simulation,Nintendo EPD,Nintendo
2,3,Super Smash Bros. Ultimate,31.09 million[4],"March 31, 2023","December 7, 2018",Fighting,Bandai Namco StudiosSora Ltd.,Nintendo
3,4,The Legend of Zelda: Breath of the Wild,29.81 million[4],"March 31, 2023","March 3, 2017",Action-adventure,Nintendo EPD,Nintendo
4,5,Pokémon Sword and Shield,25.82 million[4],"March 31, 2023","November 15, 2019",Role-playing,Game Freak,The Pokémon CompanyNintendo
...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,1 million[25],"September 8, 2020","December 20, 2018",Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo
74,72,Fitness Boxing 2: Rhythm and Exercise,1 million[26],"December 9, 2021","December 4, 2020",Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo
75,72,Shin Megami Tensei V,1 million[27],"April 18, 2022","November 11, 2021",Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo
76,72,Story of Seasons: Pioneers of Olive Town,1 million[28],"November 18, 2021","February 25, 2021",Simulationrole-playing,Marvelous,Xseed Games


In [7]:
def copies_sold_to_numeric(value):
    parsed_string = re.sub(r"million(\[[a-z]*[0-9]*\])*", "", value)
    return float(parsed_string) * 1000000

def convert_date(date):
    # month day, year
    format_type = "%B %d, %Y"  # current format
    datetime_str = datetime.datetime.strptime(date, format_type)
    return datetime_str.strftime("%m/%d/%Y") # mm/dd/yyyy

def create_metacritic_url(title):
    mod_title = re.sub(r"[.:,\']", "", title)
    mod_title = mod_title.replace(" ", "-")
    mod_title = mod_title.replace("é", "e")  # used in the pokémon title
    return "https://www.metacritic.com/game/switch/" + mod_title.lower() + "/details"

In [8]:
switch_sales_df_formatted = switch_sales_df.assign(
    Release_date = lambda df: df["Release date[a]"].map(lambda var: convert_date(var)),
    As_of = lambda df: df["As of"].map(lambda var: convert_date(var)),
    Copies_sold = lambda df: df["Copies sold"].map(lambda var: copies_sold_to_numeric(var)),
    Metacritic_url = lambda df: df["Title"].map(lambda var: create_metacritic_url(var))
).drop(columns=["Release date[a]", "As of", "Copies sold"], axis =1)

# Set the display options to show the full content of cells
pd.set_option('display.max_colwidth', None)
switch_sales_df_formatted

,Rank,Title,Genre(s),Developer(s),Publisher(s),Release_date,As_of,Copies_sold,Metacritic_url
0,1,Mario Kart 8 Deluxe,Kart racing,Nintendo EPD,Nintendo,04/28/2017,03/31/2023,53790000.0,https://www.metacritic.com/game/switch/mario-kart-8-deluxe/details
1,2,Animal Crossing: New Horizons,Social simulation,Nintendo EPD,Nintendo,03/20/2020,03/31/2023,42210000.0,https://www.metacritic.com/game/switch/animal-crossing-new-horizons/details
2,3,Super Smash Bros. Ultimate,Fighting,Bandai Namco StudiosSora Ltd.,Nintendo,12/07/2018,03/31/2023,31090000.0,https://www.metacritic.com/game/switch/super-smash-bros-ultimate/details
3,4,The Legend of Zelda: Breath of the Wild,Action-adventure,Nintendo EPD,Nintendo,03/03/2017,03/31/2023,29810000.0,https://www.metacritic.com/game/switch/the-legend-of-zelda-breath-of-the-wild/details
4,5,Pokémon Sword and Shield,Role-playing,Game Freak,The Pokémon CompanyNintendo,11/15/2019,03/31/2023,25820000.0,https://www.metacritic.com/game/switch/pokemon-sword-and-shield/details
...,...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,12/20/2018,09/08/2020,1000000.0,https://www.metacritic.com/game/switch/fitness-boxing/details
74,72,Fitness Boxing 2: Rhythm and Exercise,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,12/04/2020,12/09/2021,1000000.0,https://www.metacritic.com/game/switch/fitness-boxing-2-rhythm-and-exercise/details
75,72,Shin Megami Tensei V,Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo,11/11/2021,04/18/2022,1000000.0,https://www.metacritic.com/game/switch/shin-megami-tensei-v/details
76,72,Story of Seasons: Pioneers of Olive Town,Simulationrole-playing,Marvelous,Xseed Games,02/25/2021,11/18/2021,1000000.0,https://www.metacritic.com/game/switch/story-of-seasons-pioneers-of-olive-town/details


In [9]:
req = requests.get("https://www.metacritic.com/game/switch/mario-kart-8-deluxe/details", headers=headers)

In [10]:
def scrape_metacritic(html):
    soup = BeautifulSoup(html, "html.parser")

    description = soup.find("div", class_ = "summary_detail product_summary").find("span", class_ = "data").text
    meta_score = soup.find("div", class_ = "metascore_wrap feature_metascore").find("span").text
    meta_ratings = soup.find("div", class_ = "metascore_wrap feature_metascore").find("div", class_ = "summary").find("p").find("span", class_ = "count").find("a").find("span").text
    user_score = soup.find("div", class_ = "userscore_wrap feature_userscore").find("a", class_ = "metascore_anchor").find("div").text
    user_ratings = soup.find("div", class_ = "userscore_wrap feature_userscore").find("div", class_ = "summary").find("span", class_ = "count").text

    meta_ratings = re.findall("[0-9]+", meta_ratings)[0]
    user_ratings = re.findall("[0-9]+", user_ratings)[0]

    return {"description" : description,
                "meta_score" : float(meta_score),
                "meta_ratings" : int(meta_ratings),
                "user_score" : float(user_score),
                "user_ratings" : int(user_ratings)}
    
def perform_random_delay(delay = 3, random_offset = 1):
    time.sleep(delay + random.uniform(0, random_offset))

In [11]:
urls = switch_sales_df_formatted["Metacritic_url"].tolist()
metacritic_results = []

for url in urls:
    # go to url
    req = requests.get(url, headers=headers)
    
    # check url is successfully reached
    if req.status_code ==200:
        
        try:
            # attempt to parse
            curr_url_result = scrape_metacritic(req.text)
            curr_url_result["Metacritic_url"], curr_url_result["status_code"] = url, 200
            curr_url_result["parsed"] = True
            pass
        
        except:
            # if page cannot be scraped
            curr_url_result = {"description" : "", "meta_score" : "", "meta_ratings" : "", 
                          "user_score" : "", "user_ratings" : "", "status_code" : 200,
                          "Metacritic_url" : url, "parsed" : False}
            pass
    
    else:
        
        # page not reached
        curr_url_result = {"description" : "", "meta_score" : "", "meta_ratings" : "", 
                          "user_score" : "", "user_ratings" : "", "status_code" : req.status_code,
                          "Metacritic_url" : url, "parsed" : False}
        pass
    
    # append the result
    metacritic_results.append(curr_url_result)
    
    #perform a random delay between scrapes
    perform_random_delay()

In [12]:
reviews_df = pd.DataFrame(metacritic_results)
reviews_df

,description,meta_score,meta_ratings,user_score,user_ratings,Metacritic_url,status_code,parsed
0,"Race and battle your friends in the definitive version of Mario Kart 8. Hit the road with the definitive version of Mario Kart 8 and play anytime, anywhere! Race your friends or battle them in a revised battle mode on new and returning battle courses. Play locally in up to 4-player multiplayer in 1080p while playing in TV Mode. Every track from the Wii U version, including DLC, makes a glorious return. Plus, the Inklings appear as all-new guest characters, along with returning favorites, such as King Boo, Dry Bones, and Bowser Jr. Race your friends in the definitive version of Mario Kart 8, only on Nintendo Switch. Race as every character on every track from the Wii U version, including DLC characters and tracks. Pop some balloons in the revamped Battle mode, complete with Balloon Battle and Bob-omb Blast. Battle on new courses, like Urchin Underpass and Battle Stadium, or returning ones, such as GCN Luigi's Mansion and SNES Battle Course 1. Inkling Girl & Inkling Boy from Splatoon, King Boo, Dry Bones, and Bowser Jr. join the roster. Players can choose a new Smart Steering feature which makes driving and staying on the track easy for novice players and kids even at 200cc. Three new vehicles have been added, two are even inspired by Splatoon. Carry two items at the same time. Returning items include Boo, the item stealing ghost, and the Feather, which gives you a high jump in battle mode. Play your friends in local wireless multiplayer with up to 8 players. Drive through in 1080p HD quality in TV mode. Play on the go with handheld mode and play anytime, anywhere.",92.0,95,8.6,2698,https://www.metacritic.com/game/switch/mario-kart-8-deluxe/details,200,True
1,"If the hustle and bustle of modern life’s got you down, Tom Nook has a new business venture up his sleeve that he knows you’ll adore: the Nook Inc. Deserted Island Getaway Package. Sure, you’ve crossed paths with colorful characters near and far. Had a grand time as one of the city folk. May’ve even turned over a new leaf and dedicated yourself to public service. But deep down, isn’t there a part of you that longs for…freedom? Then perhaps a long walk on the beach of a deserted island, where a rich wealth of untouched nature awaits, is just what the doctor ordered. Peaceful creativity and charm await as you roll up your sleeves and make your new life whatever you want it to be. Collect resources and craft everything from creature comforts to handy tools. Embrace your green thumb as you interact with flowers and trees in new ways. Set up a homestead where the rules of what goes indoors and out no longer apply. Make friends with new arrivals, enjoy the seasons, pole-vault across rivers as you explore, and more.",90.0,111,5.6,6585,https://www.metacritic.com/game/switch/animal-crossing-new-horizons/details,200,True
2,"Inklings from the Splatoon series, as well as returning Smash characters like Mario and Link will be making appearances in this classic Nintendo franchise's Switch debut. Faster combat, new items, new attacks, new defensive options, and more will keep the battle raging whether you’re at home or on the go.",93.0,99,8.6,3782,https://www.metacritic.com/game/switch/super-smash-bros-ultimate/details,200,True
3,"Forget everything you know about The Legend of Zelda games. Step into a world of discovery, exploration and adventure in The Legend of Zelda: Breath of the Wild, a boundary-breaking new game in the acclaimed series. Travel across fields, through forests and to mountain peaks as you discover what has become of the ruined kingdom of Hyrule in this open-air adventure. Explore the wilds of Hyrule any way you like - Climb up towers and mountain peaks in search of new destinations, then set your own path to get there and plunge into the wilderness. Along the way, you'll battle towering enemies, hunt wild beasts and gather ingredients for the food and elixirs you'll need 

In [13]:
switch_sales_reviews_df = switch_sales_df_formatted.merge(reviews_df)
switch_sales_reviews_df

,Rank,Title,Genre(s),Developer(s),Publisher(s),Release_date,As_of,Copies_sold,Metacritic_url,description,meta_score,meta_ratings,user_score,user_ratings,status_code,parsed
0,1,Mario Kart 8 Deluxe,Kart racing,Nintendo EPD,Nintendo,04/28/2017,03/31/2023,53790000.0,https://www.metacritic.com/game/switch/mario-kart-8-deluxe/details,"Race and battle your friends in the definitive version of Mario Kart 8. Hit the road with the definitive version of Mario Kart 8 and play anytime, anywhere! Race your friends or battle them in a revised battle mode on new and returning battle courses. Play locally in up to 4-player multiplayer in 1080p while playing in TV Mode. Every track from the Wii U version, including DLC, makes a glorious return. Plus, the Inklings appear as all-new guest characters, along with returning favorites, such as King Boo, Dry Bones, and Bowser Jr. Race your friends in the definitive version of Mario Kart 8, only on Nintendo Switch. Race as every character on every track from the Wii U version, including DLC characters and tracks. Pop some balloons in the revamped Battle mode, complete with Balloon Battle and Bob-omb Blast. Battle on new courses, like Urchin Underpass and Battle Stadium, or returning ones, such as GCN Luigi's Mansion and SNES Battle Course 1. Inkling Girl & Inkling Boy from Splatoon, King Boo, Dry Bones, and Bowser Jr. join the roster. Players can choose a new Smart Steering feature which makes driving and staying on the track easy for novice players and kids even at 200cc. Three new vehicles have been added, two are even inspired by Splatoon. Carry two items at the same time. Returning items include Boo, the item stealing ghost, and the Feather, which gives you a high jump in battle mode. Play your friends in local wireless multiplayer with up to 8 players. Drive through in 1080p HD quality in TV mode. Play on the go with handheld mode and play anytime, anywhere.",92.0,95,8.6,2698,200,True
1,2,Animal Crossing: New Horizons,Social simulation,Nintendo EPD,Nintendo,03/20/2020,03/31/2023,42210000.0,https://www.metacritic.com/game/switch/animal-crossing-new-horizons/details,"If the hustle and bustle of modern life’s got you down, Tom Nook has a new business venture up his sleeve that he knows you’ll adore: the Nook Inc. Deserted Island Getaway Package. Sure, you’ve crossed paths with colorful characters near and far. Had a grand time as one of the city folk. May’ve even turned over a new leaf and dedicated yourself to public service. But deep down, isn’t there a part of you that longs for…freedom? Then perhaps a long walk on the beach of a deserted island, where a rich wealth of untouched nature awaits, is just what the doctor ordered. Peaceful creativity and charm await as you roll up your sleeves and make your new life whatever you want it to be. Collect resources and craft everything from creature comforts to handy tools. Embrace your green thumb as you interact with flowers and trees in new ways. Set up a homestead where the rules of what goes indoors and out no longer apply. Make friends with new arrivals, enjoy the seasons, pole-vault across rivers as you explore, and more.",90.0,111,5.6,6585,200,True
2,3,Super Smash Bros. Ultimate,Fighting,Bandai Namco StudiosSora Ltd.,Nintendo,12/07/2018,03/31/2023,31090000.0,https://www.metacritic.com/game/switch/super-smash-bros-ultimate/details,"Inklings from the Splatoon series, as well as returning Smash characters like Mario and Link will be making appearances in this classic Nintendo franchise's Switch debut. Faster combat, new items, new attacks, new defensive options, and more will keep the battle raging whether you’re at home or on the go.",93.0,99,8.6,3782,200,True
3,4,The Legend of Zelda: Breath of the Wild,Action-adventure,Nintendo EPD,Nintendo,03/03/2017,03/31/2023,29810000.0,https://www.metacritic.com/game/switch/the-legend-of-zelda-breath-of-the-wild/details,"Forget everything you know about The Legend of Zelda games. Step into a world of discovery, 

In [14]:
sum(switch_sales_reviews_df["parsed"]) / switch_sales_reviews_df.shape[0]

0.8846153846153846

In [15]:
# Assuming switch_sales_reviews_df is the DataFrame you mentioned

# Retrieve the rows where the "parsed" column has a value of 0
filtered_df = switch_sales_reviews_df[switch_sales_reviews_df["parsed"] == 0]

# Display the filtered DataFrame
print(filtered_df)


    Rank                                               Title   
4      5                            Pokémon Sword and Shield  \
6      7                          Pokémon Scarlet and Violet   
10    11    Pokémon: Let's Go, Pikachu! and Let's Go, Eevee!   
11    12         Pokémon Brilliant Diamond and Shining Pearl   
33    34  Momotaro Dentetsu: Showa, Heisei, Reiwa Mo Teiban!   
61    62               Nintendo Labo Toy-Con 01: Variety Kit   
65    66     Naruto Shippudden: Ultimate Ninja Storm Trilogy   
74    72               Fitness Boxing 2: Rhythm and Exercise   
77    72                                     Thief Simulator   

            Genre(s)   Developer(s)                   Publisher(s)   
4       Role-playing     Game Freak    The Pokémon CompanyNintendo  \
6       Role-playing     Game Freak    The Pokémon CompanyNintendo   
10      Role-playing     Game Freak    The Pokémon CompanyNintendo   
11      Role-playing           ILCA    The Pokémon CompanyNintendo   
33       

In [16]:
# Assuming filtered_df is the DataFrame you want to export

# Export the DataFrame as a CSV file
switch_sales_reviews_df.to_csv('data/scrapedata.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'data'